# Cross means

The compare means t-test is used to compare the mean of a variable in one group to the mean of the same variable in one, or more, other groups. The null hypothesis for the difference between the groups in the population is set to zero. We test this hypothesis using sample data.

In [1]:
import matplotlib as mpl
import pyrsm as rsm

# increase plot resolution
mpl.rcParams["figure.dpi"] = 150

In [2]:
rsm.load_data(pkg="basics", name="salary", dct=globals())

In [3]:
rsm.describe(salary)

## Salaries for Professors

### Description

The 2008-09 nine-month academic salary for Assistant Professors, Associate Professors and Professors in a college in the U.S. The data were collected as part of the on-going effort of the college's administration to monitor salary differences between male and female faculty members. A data frame with 397 observations on the following 6 variables.

### Variables

- rank = a factor with levels AsstProf, AssocProf, and Prof
- discipline = a factor with levels A ('theoretical' departments) or B ('applied' departments)
- yrs.since.phd = years since PhD
- yrs.service = years of service
- sex = a factor with levels Female and Male
- salary = nine-month salary, in dollars

### Source

Fox J. and Weisberg, S. (2011) An R Companion to Applied Regression, Second Edition Sage.


In [4]:
salary

,salary,rank,discipline,yrs.since.phd,yrs.service,sex
0,139750,Prof,B,19,18,Male
1,173200,Prof,B,20,16,Male
2,79750,AsstProf,B,4,3,Male
3,115000,Prof,B,45,39,Male
4,141500,Prof,B,40,41,Male
...,...,...,...,...,...,...
392,103106,Prof,A,33,30,Male
393,150564,Prof,A,31,19,Male
394,101738,Prof,A,42,25,Male
395,95329,Prof,A,25,15,Male


In [5]:
from typing import List, Tuple
import pandas as pd
import numpy as np
from math import sqrt
from scipy import stats

In [6]:
class CompareMeans:
    def __init__(
        self,
        data: pd.DataFrame,
        var1: str,
        var2: str,
        combinations: List[Tuple[str, str]],
        alt_hypo: str,
        conf: float,
        sample_type: str = "independent",
        multiple_comp_adjustment: str = "none",
        test_type: str = "t-test",
    ) -> None:
        self.data = data
        self.var1 = var1
        self.var2 = var2
        self.combinations = combinations
        self.alt_hypo = alt_hypo
        self.conf = conf
        self.sample_type = sample_type
        self.multiple_comp_adjustment = multiple_comp_adjustment
        self.test_type = test_type

        self.t_val = None
        self.p_val = None

        print(f"Pairwise mean comparisons {self.test_type}")

    def welch_dof(self, v1: str, v2: str) -> float:
        x = self.data[self.data[self.var1] == v1][self.var2]
        y = self.data[self.data[self.var1] == v2][self.var2]
        dof = (x.var() / x.size + y.var() / y.size) ** 2 / (
            (x.var() / x.size) ** 2 / (x.size - 1)
            + (y.var() / y.size) ** 2 / (y.size - 1)
        )

        return dof

    def calculate(self) -> None:
        combinations_elements = set()
        for combination in self.combinations:
            combinations_elements.add(combination[0])
            combinations_elements.add(combination[1])
        combinations_elements = list(combinations_elements)

        print(f"Combinations elements: {combinations_elements}")

        rows1 = []
        mean = 0
        for element in combinations_elements:
            subset = self.data[self.data[self.var1] == element][self.var2]
            row = []
            mean = np.nanmean(subset)
            n = len(subset)
            n_missing = subset.isna().sum()
            sd = subset.std()
            se = subset.sem()

            z_score = stats.norm.ppf((1 + self.conf) / 2)
            me = z_score * sd / sqrt(n)
            row = [element, mean, n, n_missing, sd, se, me]
            rows1.append(row)

        self.table1 = pd.DataFrame(
            rows1, columns=["rank", "mean", "n", "n_missing", "sd", "se", "me"]
        )

        alt_hypo_sign = " > "
        if self.alt_hypo == "less":
            alt_hypo_sign = " < "
        elif self.alt_hypo == "two-sided":
            alt_hypo_sign = " != "

        rows2 = []
        for v1, v2 in self.combinations:
            null_hypo = v1 + " = " + v2
            alt_hypo = v1 + alt_hypo_sign + v2
            diff = np.nanmean(
                self.data[self.data[self.var1] == v1][self.var2]
            ) - np.nanmean(self.data[self.data[self.var1] == v2][self.var2])

            result = stats.ttest_ind(
                self.data[self.data[self.var1] == v1][self.var2],
                self.data[self.data[self.var1] == v2][self.var2],
                equal_var=False,
                nan_policy="omit",
                alternative=self.alt_hypo,
            )

            self.t_val, self.p_val = result.statistic, result.pvalue
            se = self.data[self.data[self.var1] == v2][self.var2].sem()
            df = self.welch_dof(v1, v2)

            '''
            Not entirely sure how to calculate these
            '''
            # zero_percent = mean - stats.t.ppf(1, df) * se
            # x_percent = mean - stats.t.ppf(self.conf, df) * se

            row = [
                null_hypo,
                alt_hypo,
                diff,
                self.p_val,
                self.t_val,
                df,
                # zero_percent,
                # x_percent,
            ]
            rows2.append(row)

        self.table2 = pd.DataFrame(
            rows2,
            columns=[
                "Null hyp.",
                "Alt. hyp.",
                "diff",
                "p.value",
                "t.value",
                "df",
                # "0%",
                # str(self.conf * 100) + "%",
            ],
        )

    def summary(self) -> None:
        if self.t_val == None:
            self.calculate()
        data_name = ""
        if hasattr(self.data, "description"):
            data_name = self.data.description.split("\n")[0].split()[1].lower()
        if len(data_name) > 0:
            print(f"Data: {data_name}")
        print(f"Variables: {self.var1}, {self.var2}")
        print(f"Confidence: {self.conf}")
        print(f"Adjustment: {self.multiple_comp_adjustment}")

        print(self.table1.to_string(index=False))
        print(self.table2.to_string(index=False))


In [7]:
compare_means = CompareMeans(salary, "rank", "salary", [("AsstProf", "AssocProf"), ("AsstProf", "Prof"), ("AssocProf", "Prof")], "less", 0.95)
compare_means.calculate()
compare_means.summary()

Pairwise mean comparisons t-test
Combinations elements: ['Prof', 'AsstProf', 'AssocProf']
Data: salaries
Variables: rank, salary
Confidence: 0.95
Adjustment: none
     rank          mean   n  n_missing           sd          se          me
     Prof 126772.109023 266          0 27718.674999 1699.541008 3331.039166
 AsstProf  80775.985075  67          0  8174.112637  998.626799 1957.272560
AssocProf  93876.437500  64          0 13831.699844 1728.962480 3388.704192
           Null hyp.            Alt. hyp.          diff      p.value    t.value         df
AsstProf = AssocProf AsstProf < AssocProf -13100.452425 1.148665e-09  -6.561253 101.285901
     AsstProf = Prof      AsstProf < Prof -45996.123948 1.120135e-71 -23.333875 324.340365
    AssocProf = Prof     AssocProf < Prof -32895.671523 1.954314e-30 -13.568542 199.325499
